In software development, **refactoring** is the process of restructuring existing computer code – changing its internal structure without changing its external behavior. The goal is to improve the code's design, readability, and maintainability, making it easier to understand and modify in the future.

Think of it like renovating a house. You're not changing the number of rooms or the overall function of the house (the external behavior remains the same), but you might rearrange the furniture, repaint the walls, or update the plumbing to make it more comfortable, efficient, and easier to live in (improving the internal structure).

Here's a breakdown of key aspects of refactoring:

**Core Principles:**

- **Preserving External Behavior:** The most crucial aspect of refactoring is that it should not alter what the software does from an end-user perspective. The functionality remains the same.
- **Improving Internal Structure:** Refactoring focuses on making the code cleaner, better organized, and easier to understand. This includes aspects like reducing complexity, eliminating duplication, improving naming conventions, and enhancing modularity.
- **Small, Incremental Changes:** Refactoring is typically done in small, manageable steps. After each change, tests are run to ensure that the behavior of the code hasn't been broken. This iterative approach minimizes the risk of introducing bugs.

**Why Refactor?**

- **Improved Design:** Over time, code can become messy and lose its original well-thought-out structure as new features are added or changes are made under pressure. Refactoring helps to bring back a good design.
- **Increased Readability:** Clean, well-structured code is easier for developers (including the original author in the future) to understand. This reduces the time and effort required to maintain and modify the code.
- **Enhanced Maintainability:** Code that is easy to understand is also easier to maintain. Bugs can be found and fixed more quickly, and new features can be added with less risk of introducing unintended side effects.
- **Reduced Complexity:** Refactoring can break down large, complex methods or classes into smaller, more manageable units. This makes the code easier to reason about and test.
- **Elimination of Duplication:** Duplicated code makes maintenance harder and increases the risk of inconsistencies. Refactoring helps to identify and eliminate such redundancies.
- **Easier Testing:** Well-structured and modular code is generally easier to test. Refactoring can improve the testability of the codebase.
- **Faster Development in the Long Run:** While refactoring might take some time upfront, it can significantly speed up development in the long run by making the codebase more manageable and reducing the time spent debugging and understanding complex code.
- **Reduced Technical Debt:** Technical debt refers to the accumulated compromises and shortcuts made during development that can hinder future progress. Refactoring helps to pay down this debt.

**When to Refactor:**

- **The Rule of Three:** The first time you do something, just do it. The second time you do something similar, cringe at the duplication but do it anyway. The third time you do something similar, refactor.
- **When Adding New Features:** Before adding a new feature, it's often beneficial to refactor the existing code to make it easier to integrate the new functionality.
- **When Fixing Bugs:** Understanding the code around a bug can be challenging if it's poorly structured. Refactoring can make the bug easier to find and fix.
- **During Code Reviews:** Code reviews can often highlight areas that could benefit from refactoring.
- **When Code Smells:** Certain characteristics in code, known as "code smells," can indicate the need for refactoring. Examples include long methods, large classes, duplicate code, and poor naming.
- **Continuously:** Some development methodologies, like Extreme Programming (XP), advocate for continuous refactoring as part of the daily development process.

**Common Refactoring Techniques:**

There are numerous specific refactoring techniques, each addressing a particular code smell or aiming to improve a specific aspect of the code structure. Some common examples include:

- **Extract Method:** Turning a code fragment into a new method.
- **Rename Variable/Method/Class:** Changing the name to better reflect its purpose.
- **Move Method/Field:** Moving a method or field to another class where it logically belongs.
- **Extract Class:** Creating a new class to encapsulate related responsibilities.
- **Inline Method:** Replacing a method call with the method's body.
- **Replace Conditional with Polymorphism:** Using inheritance and polymorphism to handle different cases instead of complex conditional statements.
- **Introduce Parameter Object:** Replacing multiple related parameters with a single object.

**Tools for Refactoring:**

Modern Integrated Development Environments (IDEs) often provide built-in support for many common refactoring techniques, automating the process and ensuring that the changes are applied consistently throughout the codebase.

In summary, refactoring is a vital practice in software development that helps to maintain a healthy, manageable, and efficient codebase over time. It's an ongoing process that contributes significantly to the long-term success of software projects.


In [ ]:
import os
import sys
import time
import json
import base64
import sqlite3
import secrets
import string
import hashlib
import threading
import pyperclip
import qrcode
import zxcvbn
from datetime import datetime, timedelta
from getpass import getpass
from typing import Dict, List, Optional, Tuple, Union
from enum import Enum, auto
from dataclasses import dataclass
from cryptography.fernet import Fernet, InvalidToken
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import argon2
import hmac
import uuid
import socket
import platform
import webbrowser
from passlib import pwd
from passlib.context import CryptContext
import bcrypt
import pyotp
import gnupg
import logging
from logging.handlers import RotatingFileHandler
import signal
import atexit
try:
    import readline  # For better input handling
except ImportError:
    pass  # Not available on Windows

# Fix for collections.Callable in Python 3.10+
try:
    from collections.abc import Callable
except ImportError:
    from collections import Callable

# ==================== SECURITY CONFIGURATION ====================
class SecurityLevel(Enum):
    BASIC = auto()
    ENTERPRISE = auto()
    MILITARY = auto()

# Configure based on needed security level
CURRENT_SECURITY_LEVEL = SecurityLevel.MILITARY

# Security parameters that change based on level
SECURITY_PARAMS = {
    SecurityLevel.BASIC: {
        "argon2": {"time_cost": 2, "memory_cost": 32768, "parallelism": 2},
        "pbkdf2": {"iterations": 100000, "hash": hashes.SHA256()},
        "min_length": 12,
        "max_attempts": 10,
        "lockout_time": 60,
        "session_timeout": 3600
    },
    SecurityLevel.ENTERPRISE: {
        "argon2": {"time_cost": 3, "memory_cost": 65536, "parallelism": 4},
        "pbkdf2": {"iterations": 300000, "hash": hashes.SHA512()},
        "min_length": 16,
        "max_attempts": 5,
        "lockout_time": 300,
        "session_timeout": 1800
    },
    SecurityLevel.MILITARY: {
        "argon2": {"time_cost": 4, "memory_cost": 131072, "parallelism": 8},
        "pbkdf2": {"iterations": 600000, "hash": hashes.BLAKE2b(64)},
        "min_length": 20,
        "max_attempts": 3,
        "lockout_time": 900,
        "session_timeout": 900
    }
}

# Current configuration
CONFIG = SECURITY_PARAMS[CURRENT_SECURITY_LEVEL]

# ==================== DATA STRUCTURES ====================
@dataclass
class PasswordEntry:
    id: str
    service: str
    username: str
    encrypted_password: bytes
    notes: str
    created_at: datetime
    last_used: Optional[datetime]
    strength_score: int
    length: int
    complexity: str
    tags: List[str]
    history: List[Tuple[datetime, bytes]]  # Previous passwords with timestamps

@dataclass
class UserAccount:
    username: str
    encrypted_master: bytes
    salt: bytes
    last_login: Optional[datetime]
    failed_attempts: int
    locked_until: Optional[datetime]
    mfa_secret: Optional[str]
    permissions: List[str]

# ==================== CRYPTOGRAPHY SERVICE ====================
class CryptoService:
    @staticmethod
    def generate_key_from_password(password: str, salt: bytes) -> bytes:
        """Derive encryption key using PBKDF2"""
        kdf = PBKDF2HMAC(
            algorithm=CONFIG["pbkdf2"]["hash"],
            length=32,
            salt=salt,
            iterations=CONFIG["pbkdf2"]["iterations"],
            backend=default_backend()
        )
        return base64.urlsafe_b64encode(kdf.derive(password.encode()))

    @staticmethod
    def encrypt_data(data: str, key: bytes) -> bytes:
        """Encrypt data using Fernet (AES-128-CBC with PKCS7 padding)"""
        f = Fernet(key)
        return f.encrypt(data.encode())

    @staticmethod
    def decrypt_data(encrypted_data: bytes, key: bytes) -> str:
        """Decrypt data using Fernet"""
        f = Fernet(key)
        return f.decrypt(encrypted_data).decode()

    @staticmethod
    def generate_secure_random(length: int = 32) -> bytes:
        """Generate cryptographically secure random bytes"""
        return secrets.token_bytes(length)

    @staticmethod
    def hash_password(password: str, salt: bytes) -> str:
        """Create password hash using Argon2"""
        hasher = argon2.PasswordHasher(
            time_cost=CONFIG["argon2"]["time_cost"],
            memory_cost=CONFIG["argon2"]["memory_cost"],
            parallelism=CONFIG["argon2"]["parallelism"],
            hash_len=32,
            salt_len=16
        )
        return hasher.hash(password.encode() + salt)

    @staticmethod
    def verify_password(stored_hash: str, password: str, salt: bytes) -> bool:
        """Verify password against stored hash"""
        hasher = argon2.PasswordHasher()
        try:
            return hasher.verify(stored_hash, password.encode() + salt)
        except:
            return False

    @staticmethod
    def generate_mfa_secret() -> str:
        """Generate TOTP secret for multi-factor auth"""
        return pyotp.random_base32()

    @staticmethod
    def verify_mfa_code(secret: str, code: str) -> bool:
        """Verify TOTP code"""
        totp = pyotp.TOTP(secret)
        return totp.verify(code)

# ==================== LOGGING SERVICE ====================
class AuditLogger:
    def __init__(self):
        self.logger = logging.getLogger("PasswordManagerAudit")
        self.logger.setLevel(logging.INFO)
        
        # Create logs directory if not exists
        os.makedirs("logs", exist_ok=True)
        
        # 1MB log files, keep 5 backups
        handler = RotatingFileHandler(
            "logs/audit.log",
            maxBytes=1_000_000,
            backupCount=5,
            encoding="utf-8"
        )
        formatter = logging.Formatter(
            "%(asctime)s - %(levelname)s - %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
        
        # Also log to console in debug mode
        if "--debug" in sys.argv:
            console = logging.StreamHandler()
            console.setFormatter(formatter)
            self.logger.addHandler(console)
    
    def log_event(self, event: str, user: Optional[str] = None, level: str = "INFO"):
        """Log security events with metadata"""
        metadata = {
            "user": user,
            "ip": self._get_ip_address(),
            "host": platform.node(),
            "os": platform.platform(),
            "timestamp": datetime.utcnow().isoformat()
        }
        
        log_entry = {
            "event": event,
            "metadata": metadata,
            "level": level
        }
        
        getattr(self.logger, level.lower())(json.dumps(log_entry))
    
    def _get_ip_address(self) -> str:
        """Get local IP address for logging"""
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
        except:
            return "unknown"

# ==================== PASSWORD GENERATOR ====================
class AdvancedPasswordGenerator:
    @staticmethod
    def generate(
        length: int = 16,
        use_upper: bool = True,
        use_digits: bool = True,
        use_symbols: bool = True,
        avoid_ambiguous: bool = True,
        avoid_similar: bool = True,
        require_each_type: bool = True,
        custom_chars: str = "",
        passphrase: bool = False,
        word_count: int = 6,
        separator: str = "-"
    ) -> str:
        """
        Generate secure password or passphrase with advanced options
        
        Args:
            length: Desired length (8-256)
            passphrase: Generate memorable passphrase instead
            word_count: Number of words for passphrase
            separator: Word separator for passphrase
        """
        if passphrase:
            return AdvancedPasswordGenerator._generate_passphrase(word_count, separator)
        
        # Validate length
        if not 8 <= length <= 256:
            raise ValueError("Password length must be between 8 and 256")
        
        # Build character set
        chars = string.ascii_lowercase
        char_sets = []
        
        if use_upper:
            chars += string.ascii_uppercase
            char_sets.append(string.ascii_uppercase)
        if use_digits:
            chars += string.digits
            char_sets.append(string.digits)
        if use_symbols:
            chars += string.punctuation
            char_sets.append(string.punctuation)
        if custom_chars:
            chars += custom_chars
            char_sets.append(custom_chars)
        
        # Remove ambiguous characters if requested
        if avoid_ambiguous:
            ambiguous = 'l1IoO0'
            chars = ''.join(c for c in chars if c not in ambiguous)
            char_sets = [''.join(c for c in s if c not in ambiguous) for s in char_sets]
        
        # Remove similar characters if requested
        if avoid_similar:
            similar = 'ij1l|'
            chars = ''.join(c for c in chars if c not in similar)
            char_sets = [''.join(c for c in s if c not in similar) for s in char_sets]
        
        # Generate password with required character types
        for _ in range(100):  # Try 100 times to meet requirements
            password = ''.join(secrets.choice(chars) for _ in range(length))
            
            # Check if we need to enforce character types
            if not require_each_type:
                return password
                
            # Verify all required character types are present
            checks_passed = True
            for char_set in char_sets:
                if not any(c in char_set for c in password):
                    checks_passed = False
                    break
            
            if checks_passed:
                return password
        
        raise RuntimeError("Failed to generate password meeting requirements")

    @staticmethod
    def _generate_passphrase(word_count: int = 6, separator: str = "-") -> str:
        """Generate memorable passphrase using diceware method"""
        try:
            wordlist = AdvancedPasswordGenerator._load_wordlist()
            words = [secrets.choice(wordlist) for _ in range(word_count)]
            return separator.join(words)
        except Exception as e:
            raise RuntimeError(f"Failed to generate passphrase: {str(e)}")

    @staticmethod
    def _load_wordlist() -> List[str]:
        """Load diceware wordlist"""
        try:
            with open("diceware.wordlist", "r", encoding="utf-8") as f:
                return [line.split()[1] for line in f if line.strip()]
        except:
            # Fallback to built-in wordlist
            return [
                "correct", "horse", "battery", "staple", "secure", "password",
                "generator", "system", "privacy", "encryption", "security"
            ]

# ==================== PASSWORD MANAGER CORE ====================
class MilitaryGradePasswordManager:
    def __init__(self):
        self.db_name = "vault.db"
        self.key_file = "master.key"
        self.audit_log = AuditLogger()
        self.current_user = None
        self.user_key = None
        self.session_expiry = None
        
        # Initialize security
        self._initialize_keystore()
        self._initialize_database()
        
        # Register cleanup handlers
        atexit.register(self._cleanup)
        signal.signal(signal.SIGINT, self._handle_signal)
        signal.signal(signal.SIGTERM, self._handle_signal)
    
    def _initialize_keystore(self):
        """Initialize encryption keystore"""
        if not os.path.exists(self.key_file):
            with open(self.key_file, "wb") as f:
                f.write(CryptoService.generate_secure_random(64))
        
        # Set restrictive permissions
        if os.name != 'nt':
            os.chmod(self.key_file, 0o600)
    
    def _initialize_database(self):
        """Initialize secure database schema"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # User accounts table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    id TEXT PRIMARY KEY,
                    username TEXT UNIQUE NOT NULL,
                    encrypted_master TEXT NOT NULL,
                    salt TEXT NOT NULL,
                    last_login TEXT,
                    failed_attempts INTEGER DEFAULT 0,
                    locked_until TEXT,
                    mfa_secret TEXT,
                    permissions TEXT,
                    created_at TEXT DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            # Password vault table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS passwords (
                    id TEXT PRIMARY KEY,
                    user_id TEXT NOT NULL,
                    service TEXT NOT NULL,
                    username TEXT,
                    encrypted_password TEXT NOT NULL,
                    notes TEXT,
                    tags TEXT,
                    strength_score INTEGER,
                    length INTEGER,
                    complexity TEXT,
                    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
                    last_used TEXT,
                    history TEXT,
                    FOREIGN KEY(user_id) REFERENCES users(id)
                )
            ''')
            
            # Security events table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS security_events (
                    id TEXT PRIMARY KEY,
                    user_id TEXT,
                    event_type TEXT NOT NULL,
                    details TEXT,
                    ip_address TEXT,
                    user_agent TEXT,
                    timestamp TEXT DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            conn.commit()
    
    def _cleanup(self):
        """Cleanup resources on exit"""
        if self.user_key:
            # Securely wipe key from memory
            for i in range(len(self.user_key)):
                self.user_key[i] = 0
            self.user_key = None
        
        self.current_user = None
        self.session_expiry = None
    
    def _handle_signal(self, signum, frame):
        """Handle termination signals"""
        self._cleanup()
        sys.exit(0)
    
    def _check_session(self):
        """Verify session is still valid"""
        if not self.session_expiry or datetime.utcnow() > self.session_expiry:
            self.audit_log.log_event("Session expired", self.current_user)
            raise SessionExpiredError("Your session has expired. Please log in again.")
    
    def register_user(self, username: str, password: str, admin: bool = False):
        """Register a new user account"""
        if not username or not password:
            raise ValueError("Username and password are required")
        
        if len(password) < CONFIG["min_length"]:
            raise ValueError(f"Password must be at least {CONFIG['min_length']} characters")
        
        # Generate salt and derive key
        salt = CryptoService.generate_secure_random(16)
        master_key = CryptoService.generate_key_from_password(password, salt)
        
        # Create a second key to encrypt the master key
        storage_key = self._get_storage_key()
        encrypted_master = CryptoService.encrypt_data(base64.b64encode(master_key).decode(), storage_key)
        
        # Generate MFA secret
        mfa_secret = CryptoService.generate_mfa_secret()
        
        # Set permissions
        permissions = ["admin"] if admin else ["user"]
        
        # Create user record
        user_id = str(uuid.uuid4())
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO users (
                    id, username, encrypted_master, salt, mfa_secret, permissions
                ) VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                user_id,
                username,
                encrypted_master,
                base64.b64encode(salt).decode(),
                mfa_secret,
                json.dumps(permissions)
            ))
            conn.commit()
        
        self.audit_log.log_event("New user registered", username)
        return {
            "user_id": user_id,
            "mfa_secret": mfa_secret,
            "mfa_qr": self._generate_mfa_qr(username, mfa_secret)
        }
    
    def _get_storage_key(self) -> bytes:
        """Get key for encrypting master keys"""
        with open(self.key_file, "rb") as f:
            key_data = f.read()
        return key_data[:32]  # Use first 32 bytes
    
    def _generate_mfa_qr(self, username: str, secret: str) -> str:
        """Generate QR code for MFA setup"""
        uri = pyotp.totp.TOTP(secret).provisioning_uri(username, issuer_name="MilitaryGradePasswordManager")
        qr = qrcode.QRCode(version=1, box_size=10, border=4)
        qr.add_data(uri)
        qr.make(fit=True)
        
        img = qr.make_image(fill_color="black", back_color="white")
        filename = f"mfa_{username}_{datetime.now().strftime('%Y%m%d')}.png"
        img.save(filename)
        return filename
    
    def authenticate(self, username: str, password: str, mfa_code: Optional[str] = None):
        """Authenticate user with password and optional MFA"""
        # Check if user exists and isn't locked
        user = self._get_user(username)
        if not user:
            raise AuthenticationError("Invalid username or password")
        
        if user.locked_until and datetime.utcnow() < datetime.fromisoformat(user.locked_until):
            remaining = (datetime.fromisoformat(user.locked_until) - datetime.utcnow()).seconds // 60
            raise AccountLockedError(f"Account locked. Try again in {remaining} minutes.")
        
        # Verify password
        salt = base64.b64decode(user.salt)
        storage_key = self._get_storage_key()
        
        try:
            encrypted_master = base64.b64decode(user.encrypted_master)
            master_key_encoded = CryptoService.decrypt_data(encrypted_master, storage_key)
            master_key = base64.b64decode(master_key_encoded)
        except InvalidToken:
            raise AuthenticationError("Invalid encryption key")
        
        # Verify MFA if enabled
        if user.mfa_secret and not mfa_code:
            raise MFARequiredError("Multi-factor authentication required")
        
        if user.mfa_secret and mfa_code:
            if not CryptoService.verify_mfa_code(user.mfa_secret, mfa_code):
                self._record_failed_attempt(username)
                raise AuthenticationError("Invalid MFA code")
        
        # Update user record
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                UPDATE users 
                SET last_login = ?, failed_attempts = 0, locked_until = NULL
                WHERE username = ?
            ''', (datetime.utcnow().isoformat(), username))
            conn.commit()
        
        # Set session
        self.current_user = username
        self.user_key = master_key
        self.session_expiry = datetime.utcnow() + timedelta(seconds=CONFIG["session_timeout"])
        
        self.audit_log.log_event("User logged in", username)
        return True
    
    def _get_user(self, username: str) -> Optional[UserAccount]:
        """Retrieve user account"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT username, encrypted_master, salt, last_login, 
                       failed_attempts, locked_until, mfa_secret, permissions
                FROM users 
                WHERE username = ?
            ''', (username,))
            row = cursor.fetchone()
        
        if not row:
            return None
            
        return UserAccount(
            username=row[0],
            encrypted_master=row[1],
            salt=row[2],
            last_login=datetime.fromisoformat(row[3]) if row[3] else None,
            failed_attempts=row[4],
            locked_until=datetime.fromisoformat(row[5]) if row[5] else None,
            mfa_secret=row[6],
            permissions=json.loads(row[7])
        )
    
    def _record_failed_attempt(self, username: str):
        """Record failed login attempt and lock account if needed"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            # Increment failed attempts
            cursor.execute('''
                UPDATE users 
                SET failed_attempts = failed_attempts + 1
                WHERE username = ?
            ''', (username,))
            
            # Check if we should lock the account
            cursor.execute('''
                SELECT failed_attempts FROM users WHERE username = ?
            ''', (username,))
            attempts = cursor.fetchone()[0]
            
            if attempts >= CONFIG["max_attempts"]:
                lockout_time = datetime.utcnow() + timedelta(seconds=CONFIG["lockout_time"])
                cursor.execute('''
                    UPDATE users 
                    SET locked_until = ?
                    WHERE username = ?
                ''', (lockout_time.isoformat(), username))
                
                self.audit_log.log_event(
                    "Account locked due to failed attempts", 
                    username,
                    "WARNING"
                )
            
            conn.commit()
    
    def generate_password(self, **kwargs) -> Dict:
        """Generate secure password with analysis"""
        self._check_session()
        
        password = AdvancedPasswordGenerator.generate(**kwargs)
        analysis = self._analyze_password(password)
        
        return {
            "password": password,
            "analysis": analysis
        }
    
    def _analyze_password(self, password: str) -> Dict:
        """Comprehensive password analysis"""
        result = zxcvbn.zxcvbn(password)
        
        # Additional entropy calculation
        char_set = 0
        if any(c.islower() for c in password):
            char_set += 26
        if any(c.isupper() for c in password):
            char_set += 26
        if any(c.isdigit() for c in password):
            char_set += 10
        if any(c in string.punctuation for c in password):
            char_set += 32
        
        length = len(password)
        entropy = length * (char_set ** 0.5) if char_set > 0 else 0
        
        return {
            "length": length,
            "unique_chars": len(set(password)),
            "entropy": entropy,
            "strength": result["score"],
            "crack_time": result["crack_times_display"]["offline_slow_hashing_1e4_per_second"],
            "feedback": result["feedback"]
        }
    
    def store_password(self, service: str, password: str, **metadata):
        """Store password in encrypted vault"""
        self._check_session()
        
        if not service or not password:
            raise ValueError("Service and password are required")
        
        # Encrypt the password
        encrypted_pw = CryptoService.encrypt_data(password, self.user_key)
        
        # Create password entry
        entry_id = str(uuid.uuid4())
        analysis = self._analyze_password(password)
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                INSERT INTO passwords (
                    id, user_id, service, username, encrypted_password,
                    notes, tags, strength_score, length, complexity
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                entry_id,
                self._get_user_id(self.current_user),
                service,
                metadata.get("username"),
                encrypted_pw,
                metadata.get("notes"),
                json.dumps(metadata.get("tags", [])),
                analysis["strength"],
                len(password),
                self._get_complexity_description(password)
            ))
            
            conn.commit()
        
        self.audit_log.log_event(f"Password stored for {service}", self.current_user)
        return entry_id
    
    def _get_user_id(self, username: str) -> str:
        """Get user ID from username"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT id FROM users WHERE username = ?
            ''', (username,))
            return cursor.fetchone()[0]
    
    def _get_complexity_description(self, password: str) -> str:
        """Generate human-readable complexity description"""
        has_upper = any(c.isupper() for c in password)
        has_lower = any(c.islower() for c in password)
        has_digit = any(c.isdigit() for c in password)
        has_symbol = any(c in string.punctuation for c in password)
        
        parts = []
        if has_upper and has_lower:
            parts.append("mixed case")
        elif has_upper:
            parts.append("uppercase")
        elif has_lower:
            parts.append("lowercase")
            
        if has_digit:
            parts.append("digits")
        if has_symbol:
            parts.append("symbols")
            
        return ", ".join(parts) if parts else "simple"
    
    def get_password(self, service: str) -> Optional[PasswordEntry]:
        """Retrieve password from vault"""
        self._check_session()
        
        user_id = self._get_user_id(self.current_user)
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT id, service, username, encrypted_password, notes, tags,
                       strength_score, length, complexity, created_at, last_used, history
                FROM passwords
                WHERE user_id = ? AND service = ?
                ORDER BY last_used DESC
                LIMIT 1
            ''', (user_id, service))
            
            row = cursor.fetchone()
            
            if not row:
                return None
                
            # Update last used timestamp
            cursor.execute('''
                UPDATE passwords
                SET last_used = ?
                WHERE id = ?
            ''', (datetime.utcnow().isoformat(), row[0]))
            conn.commit()
            
            # Decrypt password
            try:
                password = CryptoService.decrypt_data(row[3], self.user_key)
            except InvalidToken:
                raise DecryptionError("Failed to decrypt password")
            
            return PasswordEntry(
                id=row[0],
                service=row[1],
                username=row[2],
                encrypted_password=password,
                notes=row[4],
                created_at=datetime.fromisoformat(row[9]),
                last_used=datetime.fromisoformat(row[10]) if row[10] else None,
                strength_score=row[6],
                length=row[7],
                complexity=row[8],
                tags=json.loads(row[5]),
                history=json.loads(row[11]) if row[11] else []
            )
    
    def export_vault(self, format: str = "json", encrypt: bool = True) -> str:
        """Export all passwords in specified format"""
        self._check_session()
        
        user_id = self._get_user_id(self.current_user)
        entries = []
        
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT service, username, encrypted_password, notes, tags
                FROM passwords
                WHERE user_id = ?
            ''', (user_id,))
            
            for row in cursor.fetchall():
                try:
                    password = CryptoService.decrypt_data(row[2], self.user_key)
                except InvalidToken:
                    continue
                
                entries.append({
                    "service": row[0],
                    "username": row[1],
                    "password": password,
                    "notes": row[3],
                    "tags": json.loads(row[4])
                })
        
        # Generate export filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"vault_export_{self.current_user}_{timestamp}"
        
        if format == "json":
            data = json.dumps(entries, indent=2)
            if encrypt:
                data = CryptoService.encrypt_data(data, self.user_key)
                filename += ".enc"
            else:
                filename += ".json"
            
            with open(filename, "wb" if encrypt else "w") as f:
                f.write(data if not encrypt else data.encode())
        
        elif format == "csv":
            import csv
            filename += ".csv"
            
            with open(filename, "w", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=["service", "username", "password", "notes", "tags"])
                writer.writeheader()
                
                for entry in entries:
                    writer.writerow({
                        "service": entry["service"],
                        "username": entry["username"],
                        "password": entry["password"],
                        "notes": entry["notes"],
                        "tags": ",".join(entry["tags"])
                    })
        
        self.audit_log.log_event(f"Vault exported to {filename}", self.current_user)
        return filename

# ==================== ERROR CLASSES ====================
class PasswordManagerError(Exception):
    """Base class for all password manager exceptions"""
    def __init__(self, message: str, code: int = 1000, details: Optional[Dict] = None):
        self.message = message
        self.code = code
        self.details = details or {}
        super().__init__(message)

    def __str__(self):
        return f"[{self.code}] {self.message}"

    def to_dict(self) -> Dict:
        """Convert error to dictionary for API responses"""
        return {
            "error": {
                "code": self.code,
                "message": self.message,
                "details": self.details
            }
        }

class AuthenticationError(PasswordManagerError):
    """Authentication related errors"""
    def __init__(self, message: str = "Authentication failed", details: Optional[Dict] = None):
        super().__init__(message, code=1100, details=details)

class AccountLockedError(AuthenticationError):
    """Account locked due to too many failed attempts"""
    def __init__(self, unlock_time: Optional[datetime] = None):
        details = {}
        if unlock_time:
            remaining = (unlock_time - datetime.utcnow()).seconds // 60
            details["unlock_in_minutes"] = remaining
            details["unlock_time"] = unlock_time.isoformat()
        
        super().__init__(
            message="Account temporarily locked due to too many failed attempts",
            code=1101,
            details=details
        )

class MFARequiredError(AuthenticationError):
    """Multi-factor authentication required"""
    def __init__(self, methods: List[str] = ["TOTP"]):
        super().__init__(
            message="Multi-factor authentication required",
            code=1102,
            details={"available_methods": methods}
        )

class MFAVerificationError(AuthenticationError):
    """MFA verification failed"""
    def __init__(self, message: str = "Invalid MFA code"):
        super().__init__(message, code=1103)

class SessionExpiredError(AuthenticationError):
    """User session has expired"""
    def __init__(self):
        super().__init__("Session expired. Please log in again", code=1104)

class AuthorizationError(PasswordManagerError):
    """Authorization/access control errors"""
    def __init__(self, message: str = "Authorization failed", details: Optional[Dict] = None):
        super().__init__(message, code=1200, details=details)

class PermissionDeniedError(AuthorizationError):
    """User lacks required permissions"""
    def __init__(self, required_permission: str, available_permissions: List[str]):
        super().__init__(
            message=f"Permission denied: {required_permission} required",
            code=1201,
            details={
                "required_permission": required_permission,
                "available_permissions": available_permissions
            }
        )

class EncryptionError(PasswordManagerError):
    """Encryption/decryption related errors"""
    def __init__(self, message: str = "Encryption/decryption error"):
        super().__init__(message, code=1300)

class DecryptionError(EncryptionError):
    """Failed to decrypt data"""
    def __init__(self, item_type: str = "password"):
        super().__init__(
            message=f"Failed to decrypt {item_type}",
            code=1301,
            details={"item_type": item_type}
        )

class KeyDerivationError(EncryptionError):
    """Failed to derive encryption key"""
    def __init__(self):
        super().__init__("Failed to derive encryption key", code=1302)

class DatabaseError(PasswordManagerError):
    """Database operation errors"""
    def __init__(self, message: str = "Database operation failed"):
        super().__init__(message, code=1400)

class IntegrityError(DatabaseError):
    """Data integrity violation"""
    def __init__(self, constraint: str):
        super().__init__(
            message=f"Data integrity violation: {constraint}",
            code=1401,
            details={"constraint": constraint}
        )

class NotFoundError(DatabaseError):
    """Requested resource not found"""
    def __init__(self, resource_type: str, resource_id: str):
        super().__init__(
            message=f"{resource_type} not found: {resource_id}",
            code=1402,
            details={
                "resource_type": resource_type,
                "resource_id": resource_id
            }
        )

class ValidationError(PasswordManagerError):
    """Data validation errors"""
    def __init__(self, message: str = "Validation failed", fields: Optional[Dict] = None):
        super().__init__(message, code=1500, details={"fields": fields or {}})

class PasswordPolicyError(ValidationError):
    """Password policy violation"""
    def __init__(self, requirements: Dict, actual: Dict):
        super().__init__(
            message="Password does not meet policy requirements",
            code=1501,
            details={
                "requirements": requirements,
                "actual": actual
            }
        )

class OperationNotAllowedError(PasswordManagerError):
    """Operation not allowed in current state"""
    def __init__(self, operation: str, reason: str):
        super().__init__(
            message=f"Operation not allowed: {operation}",
            code=1600,
            details={
                "operation": operation,
                "reason": reason
            }
        )

class RateLimitError(PasswordManagerError):
    """Rate limiting/throttling"""
    def __init__(self, retry_after: int):
        super().__init__(
            message="Too many requests. Please try again later.",
            code=1700,
            details={
                "retry_after_seconds": retry_after,
                "retry_time": (datetime.utcnow() + timedelta(seconds=retry_after)).isoformat()
            }
        )

class ConfigurationError(PasswordManagerError):
    """Configuration errors"""
    def __init__(self, message: str = "Configuration error"):
        super().__init__(message, code=1800)

class SecurityError(PasswordManagerError):
    """Security-related errors"""
    def __init__(self, message: str = "Security violation detected"):
        super().__init__(message, code=1900)

class AuditLogError(PasswordManagerError):
    """Audit logging errors"""
    def __init__(self, message: str = "Audit logging failed"):
        super().__init__(message, code=2000)

# ==================== MAIN APPLICATION ====================
if __name__ == "__main__":
    try:
        manager = MilitaryGradePasswordManager()

        print("""
        ██████╗ ██╗   ██╗ █████╗ ███╗   ██╗████████╗██╗   ██╗███╗   ███╗
        ██╔══██╗██║   ██║██╔══██╗████╗  ██║╚══██╔══╝██║   ██║████╗ ████║
        ██████╔╝██║   ██║███████║██╔██╗ ██║   ██║   ██║   ██║██╔████╔██║
        ██╔═══╝ ██║   ██║██╔══██║██║╚██╗██║   ██║   ██║   ██║██║╚██╔╝██║
        ██║     ╚██████╔╝██║  ██║██║ ╚████║   ██║   ╚██████╔╝██║ ╚═╝ ██║
        ╚═╝      ╚═════╝ ╚═╝  ╚═╝╚═╝  ╚═══╝   ╚═╝    ╚═════╝ ╚═╝     ╚═╝
        """)
        print("Quantum-Resistant Password Vault v2.0")
        print("-------------------------------------")

        # Example usage:
        # 1. Register a new user
        # registration = manager.register_user("admin", "SuperSecurePassword123!", admin=True)
        # print(f"MFA QR code saved to: {registration['mfa_qr']}")
        # print(f"MFA Secret: {registration['mfa_secret']}")
        
        # 2. Authenticate
        # manager.authenticate("admin", "SuperSecurePassword123!", "123456")
        
        # 3. Generate and store password
        # result = manager.generate_password(length=32, use_symbols=True)
        # print(f"Generated password: {result['password']}")
        # print(f"Strength: {result['analysis']['strength']}/4")
        # manager.store_password("example.com", result["password"], username="admin")
        
        # 4. Retrieve password
        # entry = manager.get_password("example.com")
        # print(f"Password for example.com: {entry.password}")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}", file=sys.stderr)
        sys.exit(1)